# 04 - Modelo Alternativo 2: Support Vector Machine (SVM)

## Descripción

Este notebook presenta una **segunda aproximación alternativa** usando **Support Vector Machine** para clasificación multiclase.

### ¿Por qué SVM?

- **Márgenes máximos**: Busca el hiperplano óptimo que maximiza la separación entre clases
- **Kernel trick**: Puede capturar relaciones no lineales mediante transformaciones implícitas
- **Efectivo en alta dimensionalidad**: Funciona bien con muchas features
- **Robusto**: Menos propenso a overfitting con regularización apropiada


---
## 1. Importaciones y Configuración

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

# Modelo SVM
from sklearn.svm import SVC

# Métricas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Semilla para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Librerías cargadas correctamente")

✓ Librerías cargadas correctamente


---
## 2. Carga de Datos

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test['ID'].copy()

print(f"Train: {train.shape}")
print(f"Test: {test.shape}")

Train: (692500, 21)
Test: (296786, 20)


---
## 3. Preprocesamiento

Aplicamos el mismo pipeline pero **con estandarización** ya que SVM es muy sensible a las escalas de las variables.

In [3]:
def preprocess_data(df, is_train=True):
    """Preprocesamiento para SVM (sin estandarización, se aplica después)"""
    df_proc = df.copy()
    
    # Eliminar duplicados
    if 'F_TIENEINTERNET.1' in df_proc.columns:
        df_proc = df_proc.drop('F_TIENEINTERNET.1', axis=1)
    
    # E_VALORMATRICULAUNIVERSIDAD
    def convertir_valor_matricula(valor):
        if pd.isna(valor): return np.nan
        elif 'Menos de 500 mil' in valor: return 250000
        elif 'Entre 500 mil y menos de 1 millón' in valor: return 750000
        elif 'Entre 1 millón y menos de 2.5 millones' in valor: return 1750000
        elif 'Entre 2.5 millones y menos de 4 millones' in valor: return 3250000
        elif 'Entre 4 millones y menos de 5.5 millones' in valor: return 4750000
        elif 'Entre 5.5 millones y menos de 7 millones' in valor: return 6250000
        elif 'Más de 7 millones' in valor: return 7500000
        elif 'No pagó matrícula' in valor: return 0
        else: return np.nan
    
    if 'E_VALORMATRICULAUNIVERSIDAD' in df_proc.columns:
        df_proc['E_VALORMATRICULAUNIVERSIDAD'] = df_proc['E_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)
        df_proc['E_VALORMATRICULAUNIVERSIDAD'].fillna(df_proc['E_VALORMATRICULAUNIVERSIDAD'].mean(), inplace=True)
    
    # E_HORASSEMANATRABAJA
    def convertir_horas_trabajadas(valor):
        if pd.isna(valor): return np.nan
        valor = str(valor)
        if valor == '0': return 0
        elif 'Menos de 10 horas' in valor: return 5
        elif 'Entre 11 y 20 horas' in valor: return 15.5
        elif 'Entre 21 y 30 horas' in valor: return 25.5
        elif 'Más de 30 horas' in valor: return 35
        return np.nan
    
    if 'E_HORASSEMANATRABAJA' in df_proc.columns:
        df_proc['E_HORASSEMANATRABAJA'] = df_proc['E_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)
        df_proc['E_HORASSEMANATRABAJA'].fillna(0, inplace=True)
    
    # F_ESTRATOVIVIENDA
    if 'F_ESTRATOVIVIENDA' in df_proc.columns:
        estrato_map = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 
                       'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
        df_proc['F_ESTRATOVIVIENDA'] = df_proc['F_ESTRATOVIVIENDA'].map(estrato_map)
        df_proc['F_ESTRATOVIVIENDA'].fillna(0, inplace=True)
    
    # Variables binarias
    binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 
                   'F_TIENECOMPUTADOR', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO']
    for col in binary_cols:
        if col in df_proc.columns:
            df_proc[col] = df_proc[col].map({'Si': 1, 'No': 0})
            df_proc[col].fillna(0, inplace=True)
    
    # Target
    if is_train and 'RENDIMIENTO_GLOBAL' in df_proc.columns:
        target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
        df_proc['RENDIMIENTO_GLOBAL'] = df_proc['RENDIMIENTO_GLOBAL'].map(target_map)
    
    # Educación padres - One-hot
    def to_onehot(x):
        values = np.unique(x)
        indices = [np.argwhere(i == values)[0][0] for i in x]
        return np.eye(len(values))[indices].astype(int)
    
    def replace_column_with_onehot(data, col):
        values = np.unique(data[col])
        onehot_matrix = to_onehot(data[col].values)
        return pd.DataFrame(onehot_matrix, 
                          columns=[f"{col}_{values[i]}" for i in range(onehot_matrix.shape[1])], 
                          index=data.index)
    
    for col in ['F_EDUCACIONMADRE', 'F_EDUCACIONPADRE']:
        if col in df_proc.columns:
            df_proc[col].fillna('No Aplica', inplace=True)
            onehot_df = replace_column_with_onehot(df_proc[[col]], col)
            df_proc = df_proc.join(onehot_df)
            df_proc.drop(col, axis=1, inplace=True)
    
    # E_PRGM_ACADEMICO
    if 'E_PRGM_ACADEMICO' in df_proc.columns:
        le = LabelEncoder()
        df_proc['E_PRGM_ACADEMICO'] = le.fit_transform(df_proc['E_PRGM_ACADEMICO'])
    
    # E_PRGM_DEPARTAMENTO
    if 'E_PRGM_DEPARTAMENTO' in df_proc.columns:
        df_proc['E_PRGM_DEPARTAMENTO'].fillna('DESCONOCIDO', inplace=True)
        depto_onehot = replace_column_with_onehot(df_proc[['E_PRGM_DEPARTAMENTO']], 'E_PRGM_DEPARTAMENTO')
        df_proc = df_proc.join(depto_onehot)
        df_proc.drop('E_PRGM_DEPARTAMENTO', axis=1, inplace=True)
    
    # Indicadores
    indicator_cols = [col for col in df_proc.columns if col.startswith('INDICADOR_')]
    for col in indicator_cols:
        if df_proc[col].isnull().sum() > 0:
            df_proc[col].fillna(df_proc[col].median(), inplace=True)
    
    # Feature Engineering
    recursos = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR']
    recursos = [c for c in recursos if c in df_proc.columns]
    if recursos:
        df_proc['TOTAL_RECURSOS_HOGAR'] = df_proc[recursos].sum(axis=1)
    
    if indicator_cols:
        df_proc['PROMEDIO_INDICADORES'] = df_proc[indicator_cols].mean(axis=1)
    
    # Eliminar ID y PERIODO
    for col in ['ID', 'PERIODO_ACADEMICO']:
        if col in df_proc.columns:
            df_proc = df_proc.drop(col, axis=1)
    
    return df_proc

print("Preprocesando train...")
train_processed = preprocess_data(train, is_train=True)
print("\nPreprocesando test...")
test_processed = preprocess_data(test, is_train=False)
print(f"\n✓ Train: {train_processed.shape}")
print(f"✓ Test: {test_processed.shape}")

Preprocesando train...

Preprocesando test...

Preprocesando test...

✓ Train: (692500, 72)
✓ Test: (296786, 71)

✓ Train: (692500, 72)
✓ Test: (296786, 71)


In [4]:
# Separar features y target
y = train_processed['RENDIMIENTO_GLOBAL'].copy()
X = train_processed.drop('RENDIMIENTO_GLOBAL', axis=1)
X_test = test_processed.copy()

# Alinear columnas
all_cols = sorted(list(set(X.columns) | set(X_test.columns)))
X = X.reindex(columns=all_cols, fill_value=0)
X_test = X_test.reindex(columns=all_cols, fill_value=0)

print(f"✓ X: {X.shape}")
print(f"✓ X_test: {X_test.shape}")

✓ X: (692500, 71)
✓ X_test: (296786, 71)


---
## 4. División de Datos y Estandarización

**Importante**: SVM requiere estandarización para funcionar correctamente.

In [5]:
# División de datos
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Estandarización (CRUCIAL para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train_scaled.shape[0]:,}")
print(f"Validation: {X_val_scaled.shape[0]:,}")
print(f"Features: {X_train_scaled.shape[1]}")
print(f"\n✓ Datos estandarizados correctamente")

Train: 554,000
Validation: 138,500
Features: 71

✓ Datos estandarizados correctamente


---
## 5. Modelo Base SVM

Comenzamos con un modelo base usando kernel RBF (Radial Basis Function).

> **Nota**: SVM es computacionalmente costoso, por lo que usamos una muestra para la búsqueda de hiperparámetros.

In [ ]:
print("="*80)
print("SVM - MODELO BASE (kernel RBF)")
print("="*80)

# Modelo base con parámetros por defecto
svm_base = SVC(
    kernel='rbf',
    random_state=RANDOM_STATE,
    verbose=False
)

print("\nEntrenando modelo base...")
svm_base.fit(X_train_scaled, y_train)

y_train_pred_base = svm_base.predict(X_train_scaled)
y_val_pred_base = svm_base.predict(X_val_scaled)

train_acc_base = accuracy_score(y_train, y_train_pred_base)
val_acc_base = accuracy_score(y_val, y_val_pred_base)

print(f"\nAccuracy Train: {train_acc_base:.4f}")
print(f"Accuracy Validation: {val_acc_base:.4f}")
print(f"Diferencia: {train_acc_base - val_acc_base:.4f}")

SVM - MODELO BASE (kernel RBF)

Entrenando modelo base...


---
## 6. Optimización de Hiperparámetros

Usamos GridSearchCV con una muestra reducida para optimizar `C` y `gamma`.

In [ ]:
print("="*80)
print("OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("="*80)

# Usamos una muestra para acelerar la búsqueda
sample_size = min(5000, len(X_train_scaled))
sample_idx = np.random.choice(len(X_train_scaled), sample_size, replace=False)
X_sample = X_train_scaled[sample_idx]
y_sample = y_train.iloc[sample_idx]

print(f"\nUsando muestra de {sample_size:,} observaciones para optimización")

# Grid de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1],
    'kernel': ['rbf', 'poly']
}

svm_model = SVC(random_state=RANDOM_STATE)

grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

print("\nBuscando mejores hiperparámetros...")
grid_search.fit(X_sample, y_sample)

print("\n✓ Optimización completa")
print(f"\nMejores hiperparámetros:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMejor score CV: {grid_search.best_score_:.4f}")

---
## 7. Modelo Final Optimizado

In [ ]:
print("="*80)
print("SVM OPTIMIZADO - ENTRENAMIENTO COMPLETO")
print("="*80)

# Entrenar con todos los datos usando los mejores hiperparámetros
best_svm = SVC(
    **grid_search.best_params_,
    random_state=RANDOM_STATE,
    verbose=False
)

print("\nEntrenando modelo optimizado con todos los datos...")
best_svm.fit(X_train_scaled, y_train)

y_train_pred = best_svm.predict(X_train_scaled)
y_val_pred = best_svm.predict(X_val_scaled)

train_acc = accuracy_score(y_train, y_train_pred)
val_acc = accuracy_score(y_val, y_val_pred)

print(f"\nAccuracy Train: {train_acc:.4f}")
print(f"Accuracy Validation: {val_acc:.4f}")
print(f"Diferencia: {train_acc - val_acc:.4f}")

print(f"\nMejora vs modelo base:")
print(f"  Train: {train_acc - train_acc_base:+.4f}")
print(f"  Validation: {val_acc - val_acc_base:+.4f}")

---
## 8. Evaluación

In [ ]:
target_names = ['bajo', 'medio-bajo', 'medio-alto', 'alto']
print(classification_report(y_val, y_val_pred, target_names=target_names))

### Matriz de Confusión

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Matriz de Confusión - SVM', fontsize=14, fontweight='bold')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

# Accuracy por clase
print("\nAccuracy por clase:")
for i, clase in enumerate(target_names):
    class_acc = cm[i, i] / cm[i, :].sum()
    print(f"  {clase}: {class_acc:.4f}")

---
## 9. Predicciones en Test


In [ ]:
# Predicciones en test
y_test_pred = best_svm.predict(X_test_scaled)

# Convertir a etiquetas
target_map_inverse = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
y_test_labels = pd.Series(y_test_pred).map(target_map_inverse)

# Crear submission
submission_svm = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': y_test_labels
})

submission_svm.to_csv('submission_svm.csv', index=False)

print(f"✓ Predicciones generadas: {len(y_test_pred):,}")
print(f"\nDistribución en test:")
print(y_test_labels.value_counts().sort_index())
print(f"\n✓ Archivo guardado: submission_svm.csv")

---
## 10. Conclusiones

### Ventajas de SVM:
- ✅ **Márgenes óptimos**: Busca la mejor separación entre clases
- ✅ **Kernel trick**: Captura relaciones no lineales complejas
- ✅ **Efectivo en alta dimensionalidad**: Funciona bien con muchas features
- ✅ **Robusto con parámetro C apropiado**: Menos overfitting

### Desventajas:
- ❌ **Muy lento**: No escala bien a datasets grandes 
- ❌ **Requiere estandarización**: Preprocessing más cuidadoso
- ❌ **No probabilístico por defecto**: Necesita calibración para probabilidades
- ❌ **Memoria intensiva**: Guarda vectores de soporte

